
# <center> Text generation using RNN/LSTM (Character-level)</center>
This code implements a Recurrent Neural Network with LSTM/RNN units for training/sampling from character-level language models. In other words, the model takes a text file as input and trains the RNN network that learns to predict the next character in a sequence.  
The RNN can then be used to generate text character by character that will look like the original training data. 

This code is based on this [blog](http://karpathy.github.io/2015/05/21/rnn-effectiveness/), and the code is an step-by-step implimentation of the [character-level implimentation](https://github.com/crazydonkey200/tensorflow-char-rnn).




In [1]:
# install tensorflow=1.8 and restart Runtime before executiing subsequent cells
!pip install tensorflow==1.8.0

     |████████████████████████████████| 49.1MB 63kB/s 
     |████████████████████████████████| 3.1MB 57.5MB/s 
     |████████████████████████████████| 890kB 54.5MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107220 sha256=1c4a96a011217096b91e58d4457677c2f505bca742a93e39f3d216c60aa67652
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.2.1
    Uninstalling bleach-3.2.1:
      Successfully uninstalled bleach-3.2.1
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [2]:
import tensorflow as tf
import time
import codecs
import os
import collections
from six.moves import cPickle
import numpy as np

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:522: FutureWarning: Passing (type, 1) or 

### Data loader
The following cell is a class that help to read data from input file.

In [3]:
class TextLoader():
    def __init__(self, data_dir, batch_size, seq_length, encoding='utf-8'):
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.seq_length = seq_length
        self.encoding = encoding

        input_file = os.path.join(data_dir, "input.txt")
        vocab_file = os.path.join(data_dir, "vocab.pkl")
        tensor_file = os.path.join(data_dir, "data.npy")

        if not (os.path.exists(vocab_file) and os.path.exists(tensor_file)):
            print("reading text file")
            self.preprocess(input_file, vocab_file, tensor_file)
        else:
            print("loading preprocessed files")
            self.load_preprocessed(vocab_file, tensor_file)
        self.create_batches()
        self.reset_batch_pointer()

    def preprocess(self, input_file, vocab_file, tensor_file):
        with codecs.open(input_file, "r", encoding=self.encoding) as f:
            data = f.read()
        counter = collections.Counter(data)
        count_pairs = sorted(counter.items(), key=lambda x: -x[1])
        self.chars, _ = zip(*count_pairs)
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(len(self.chars))))
        with open(vocab_file, 'wb') as f:
            cPickle.dump(self.chars, f)
        self.tensor = np.array(list(map(self.vocab.get, data)))
        np.save(tensor_file, self.tensor)

    def load_preprocessed(self, vocab_file, tensor_file):
        with open(vocab_file, 'rb') as f:
            self.chars = cPickle.load(f)
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(len(self.chars))))
        self.tensor = np.load(tensor_file)
        self.num_batches = int(self.tensor.size / (self.batch_size * self.seq_length))

    def create_batches(self):
        self.num_batches = int(self.tensor.size / (self.batch_size * self.seq_length))

        # When the data (tensor) is too small, let's give them a better error message
        if self.num_batches==0:
            assert False, "Not enough data. Make seq_length and batch_size small."

        self.tensor = self.tensor[:self.num_batches * self.batch_size * self.seq_length]
        xdata = self.tensor
        ydata = np.copy(self.tensor)
        ydata[:-1] = xdata[1:]
        ydata[-1] = xdata[0]
        self.x_batches = np.split(xdata.reshape(self.batch_size, -1), self.num_batches, 1)
        self.y_batches = np.split(ydata.reshape(self.batch_size, -1), self.num_batches, 1)


    def next_batch(self):
        x, y = self.x_batches[self.pointer], self.y_batches[self.pointer]
        self.pointer += 1
        return x, y

    def reset_batch_pointer(self):
        self.pointer = 0

### Parameters
#### Batch, number_of_batch, batch_size and seq_length
what is batch, number_of_batch, batch_size and seq_length in the charcter level example?  

Lets assume the input is this sentence: '__here is an example__'. Then:
- txt_length = 18  
- seq_length = 3  
- batch_size = 2  
- number_of_batchs = 18/3*2 = 3
- batch = array (['h','e','r'],['e',' ','i'])
- sample Seq = 'her'  

In [4]:
seq_length = 50 # RNN sequence length
batch_size = 6  # 60 minibatch size, i.e. size of data in each epoch
num_epochs = 5 # 50  125,  you should change it to 50 if you want to see a relatively good results
learning_rate = 0.002
decay_rate = 0.97
rnn_size = 128 # size of RNN hidden state (output dimension)
num_layers = 2 #number of layers in the RNN

In [5]:
!wget -nv -O input.txt https://ibm.box.com/shared/static/a3f9e9mbpup09toq35ut7ke3l3lf03hg.txt 
with open('input.txt', 'r') as f:
    read_data = f.read()
    print (read_data[0:100])
f.closed

2020-11-11 05:05:12 URL:https://public.boxcloud.com/d/1/b1!rUD7nU8S-zidDJtijpCnTi2mlrxg7J9UyBaBNuDOoT8vT-jw0MNXRy6OQqBK5hbxcQqvK0X_W5lUhR0pBkJT691cYhvVJLicsrrx-w4Epj-eMv6UqeTS-FZNvsb7DrVIqRGpkPB_lzdfP2pN0xmSDlYjdED_b_mY4qVWz9GghwEb4eXbvBglNyYeCVq9h9H2r5EfMdRxeTzP61Alt4ZUZFvMUHLUKvQTlDbOSxBzAmOMAMblceKY1rVzSoXHh7JpHsjdkGsk8FySuZOqbJOEr4ofRum-a1zHUWdVOGhAxF3OBb9oopOWFF-7pYZ_I1nbKi8gspRqNSPtANVDmzj3G8IUFb4C_uKRsBV0mATEAqqagJ1W04RkiJUAZFnjAQlU5MsrDl1zzi7JbD6aqcfCg11ppX86IILAiTl-ainTwWMFZhP600yC2l9hC1vx_M568EfbZ5cLl1YseyZX38Vs1_hl1deA6MhqYq0KTNzGIVYUc-j_MpX17jYsAtXRQItyqtzfQ04oazpj_OOiialyBpED_67gBuNp5P1CMx4KvhMjTzM4yftBIp9wfONE8beZ4uQhOJcluTb7Pgwldr75mmG9i6k4qskWGn8hmnQlLK3ShNHFTcrMYGftGVmy15YPhT2FZ7C8cupOZwCGNgfTUJVH5xnGYNNnDU5RbJHnDEmxKvKwiv_W40wWeTC8DvEu90A1amAuDQqpGb-PuhSV38kWHvDbWD9ILrzurrfZ_CdwhzT_vPuEAktfMCETr6aSvwLZ_uP8SEDczsMwdrCY6Jnfxj9DZG2rlICgMy5cd9rONLdQ_6wgstxIaRIowlUZLweVoDs5I459DqaRq63eWpAkKYxaqhLNAKunS-c1sa3kR4V5GXefw2kCJvxc1q7wuWGo2o3hqcFOIF2xz8jKRH7DGrGNj_fRwK8RQPzJyTXAK

True

Now, we can read the data at batches using the __TextLoader__ class. It will convert the characters to numbers, and represent each sequence as a vector in batches:

In [6]:
data_loader = TextLoader('', batch_size, seq_length)
vocab_size = data_loader.vocab_size
print ("vocabulary size:" ,data_loader.vocab_size)
print ("Characters:" ,data_loader.chars)
print ("vocab number of 'F':",data_loader.vocab['F'])
print ("Character sequences (first batch):", data_loader.x_batches[0])

reading text file
vocabulary size: 65
Characters: (' ', 'e', 't', 'o', 'a', 'h', 's', 'r', 'n', 'i', '\n', 'l', 'd', 'u', 'm', 'y', ',', 'w', 'f', 'c', 'g', 'I', 'b', 'p', ':', '.', 'A', 'v', 'k', 'T', "'", 'E', 'O', 'N', 'R', 'S', 'L', 'C', ';', 'W', 'U', 'H', 'M', 'B', '?', 'G', '!', 'D', '-', 'F', 'Y', 'P', 'K', 'V', 'j', 'q', 'x', 'z', 'J', 'Q', 'Z', 'X', '3', '&', '$')
vocab number of 'F': 49
Character sequences (first batch): [[49  9  7  6  2  0 37  9  2  9 57  1  8 24 10 43  1 18  3  7  1  0 17  1
   0 23  7  3 19  1  1 12  0  4  8 15  0 18 13  7  2  5  1  7 16  0  5  1
   4  7]
 [15 46 10 10 47 32 34 35 31 29 24 10 47  9  6 23 13  2  1  0  8  3  2  0
  17  9  2  5  0  5  1  7 38  0  6  5  1  0  9  6  0 11 13  8  4  2  9 19
  25 10]
 [ 0  2  5  4  2  0 15  3 13  8 20  0 42  4  7  6  0  3 18  0 14  1  8 16
  10 49  7  3 14  0 18  3  7  2  5  0  2  5  1  0  7  4  8 28  6  0  3 18
   0 14]
 [11  1  8  2  0 28  8  4 27  1  0  9  6  0  2  5  9  6  0  6  4 14  1 46
  10 10 35  1 19  3

### LSTM Architecture
Each LSTM cell has 5 parts:
1. Input
2. prv_state
3. prv_output
4. new_state
5. new_output


- Each LSTM cell has an input layre, which its size is 128 units in our case. The input vector's dimension also is 128, which is the dimensionality of embedding vector, so called, dimension size of W2V/embedding, for each character/word.
- Each LSTM cell has a hidden layer, where there are some hidden units. The argument n_hidden=128 of BasicLSTMCell is the number of hidden units of the LSTM (inside A). It keeps the size of the output and state vector. It is also known as, rnn_size, num_units, num_hidden_units, and LSTM size
- An LSTM keeps two pieces of information as it propagates through time: 
    - __hidden state__ vector: Each LSTM cell accept a vector, called __hidden state__ vector, of size n_hidden=128, and its value is returned to the LSTM cell in the next step. The __hidden state__ vector; which is the memory of the LSTM, accumulates using its (forget, input, and output) gates through time. "num_units" is equivalant to "size of RNN hidden state". number of hidden units is the dimensianality of the output (= dimesianality of the state) of the LSTM cell.
    - __previous time-step output__: For each LSTM cell that we initialize, we need to supply a value (128 in this case) for the hidden dimension, or as some people like to call it, the number of units in the LSTM cell. 


#### num_layers = 2 
- number of layers in the RNN, is defined by num_layers
- An input of MultiRNNCell is __cells__ which is list of RNNCells that will be composed in this order.

The memory state of the network is initialized with a vector of zeros and gets updated after reading each character.

__BasicRNNCell.zero_state(batch_size, dtype)__ Return zero-filled state tensor(s). In this function, batch_size
representing the batch size.

### Embedding
In this section, we build a 128-dim vector for each character. As we have 60 batches, and 50 character in each sequence, it will generate a [60,50,128] matrix. The function `tf.get_variable()` is used to share a variable and to initialize it in one place. `tf.get_variable()` is used to get or create a variable instead of a direct call to `tf.Variable`. 

### Feeding a batch of 50 sequence to a RNN:

The feeding process for iputs is as following:

- Step 1:  first character of each of the 50 sentences (in a batch) is entered in parallel.  
- Step 2:  second character of each of the 50 sentences is input in parallel. 
- Step n: nth character of each of the 50 sentences is input in parallel.  

The parallelism is only for efficiency.  Each character in a batch is handled in parallel,  but the network sees one character of a sequence at a time and does the computations accordingly. All the computations involving the characters of all sequences in a batch at a given time step are done in parallel. 

In [7]:
class LSTMModel():
    def __init__(self,sample=False):
        rnn_size = 128 # size of RNN hidden state vector
        batch_size = 6 # minibatch size, i.e. size of dataset in each epoch
        seq_length = 50 # RNN sequence length
        num_layers = 2 # number of layers in the RNN
        vocab_size = 65
        grad_clip = 5.
        if sample:
            print(">> sample mode:")
            batch_size = 1
            seq_length = 1
        # The core of the model consists of an LSTM cell that processes one char at a time and computes probabilities of the possible continuations of the char. 
        basic_cell = tf.contrib.rnn.BasicRNNCell(rnn_size)
        # model.cell.state_size is (128, 128)
        self.stacked_cell = tf.contrib.rnn.MultiRNNCell([basic_cell] * num_layers)

        self.input_data = tf.placeholder(tf.int32, [batch_size, seq_length], name="input_data")
        self.targets = tf.placeholder(tf.int32, [batch_size, seq_length], name="targets")
        # Initial state of the LSTM memory.
        # The memory state of the network is initialized with a vector of zeros and gets updated after reading each char. 
        self.initial_state = self.stacked_cell.zero_state(batch_size, tf.float32) #why batch_size

        with tf.variable_scope('rnnlm_class1'):
            softmax_w = tf.get_variable("softmax_w", [rnn_size, vocab_size]) #128x65
            softmax_b = tf.get_variable("softmax_b", [vocab_size]) # 1x65
            with tf.device("/cpu:0"):
                embedding = tf.get_variable("embedding", [vocab_size, rnn_size])  #65x128
                inputs = tf.split(tf.nn.embedding_lookup(embedding, self.input_data), seq_length, 1)
                inputs = [tf.squeeze(input_, [1]) for input_ in inputs]
                #inputs = tf.split(em, seq_length, 1)
                
                


        # The value of state is updated after processing each batch of chars.
        outputs, last_state = tf.contrib.legacy_seq2seq.rnn_decoder(inputs, self.initial_state, self.stacked_cell, loop_function=None, scope='rnnlm_class1')
        output = tf.reshape(tf.concat(outputs,1), [-1, rnn_size])
        self.logits = tf.matmul(output, softmax_w) + softmax_b
        self.probs = tf.nn.softmax(self.logits)
        loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([self.logits],
                [tf.reshape(self.targets, [-1])],
                [tf.ones([batch_size * seq_length])],
                vocab_size)
        self.cost = tf.reduce_sum(loss) / batch_size / seq_length
        self.final_state = last_state
        self.lr = tf.Variable(0.0, trainable=False)
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.cost, tvars),grad_clip)
        optimizer = tf.train.AdamOptimizer(self.lr)
        self.train_op = optimizer.apply_gradients(zip(grads, tvars))
    
    
    def sample(self, sess, chars, vocab, num=200, prime='The ', sampling_type=1):
        state = sess.run(self.stacked_cell.zero_state(1, tf.float32))
        #print state
        for char in prime[:-1]:
            x = np.zeros((1, 1))
            x[0, 0] = vocab[char]
            feed = {self.input_data: x, self.initial_state:state}
            [state] = sess.run([self.final_state], feed)

        def weighted_pick(weights):
            t = np.cumsum(weights)
            s = np.sum(weights)
            return(int(np.searchsorted(t, np.random.rand(1)*s)))

        ret = prime
        char = prime[-1]
        for n in range(num):
            x = np.zeros((1, 1))
            x[0, 0] = vocab[char]
            feed = {self.input_data: x, self.initial_state:state}
            [probs, state] = sess.run([self.probs, self.final_state], feed)
            p = probs[0]

            if sampling_type == 0:
                sample = np.argmax(p)
            elif sampling_type == 2:
                if char == ' ':
                    sample = weighted_pick(p)
                else:
                    sample = np.argmax(p)
            else: # sampling_type == 1 default:
                sample = weighted_pick(p)

            pred = chars[sample]
            ret += pred
            char = pred
        return ret

In [8]:
# Creating the LSTM object
# Now we create a LSTM model:

with tf.variable_scope("rnn"):
    model = LSTMModel()

In [10]:
# Train usinng LSTMModel class
#We can train our model through feeding batches:

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for e in range(num_epochs): # num_epochs is 5 for test, but should be higher
        sess.run(tf.assign(model.lr, learning_rate * (decay_rate ** e)))
        data_loader.reset_batch_pointer()
        state = sess.run(model.initial_state) # (2x[60x128])
        for b in range(data_loader.num_batches): #for each batch
            start = time.time()
            x, y = data_loader.next_batch()
            feed = {model.input_data: x, model.targets: y, model.initial_state:state}
            train_loss, state, _ = sess.run([model.cost, model.final_state, model.train_op], feed)
            end = time.time()
        print("{}/{} (epoch {}), train_loss = {:.3f}, time/batch = {:.3f}" \
                .format(e * data_loader.num_batches + b, num_epochs * data_loader.num_batches, e, train_loss, end - start))
        with tf.variable_scope("rnn", reuse=True):
            sample_model = LSTMModel(sample=True)
            print (sample_model.sample(sess, data_loader.chars , data_loader.vocab, num=50, prime='The ', sampling_type=1))
            print ('----------------------------------')

3716/18585 (epoch 0), train_loss = 1.668, time/batch = 0.014
>> sample mode:
The malle ropo sawty then he maid they fairle. Would w
----------------------------------
7433/18585 (epoch 1), train_loss = 1.564, time/batch = 0.013
>> sample mode:
The spitte
But have begants out, and mine now
so, bust
----------------------------------
11150/18585 (epoch 2), train_loss = 1.514, time/batch = 0.014
>> sample mode:
The us my brown's
Are; and as you you uncquasiors of y
----------------------------------
14867/18585 (epoch 3), train_loss = 1.538, time/batch = 0.014
>> sample mode:
The lacious deceas, We then near? where, merrain whose
----------------------------------
18584/18585 (epoch 4), train_loss = 1.502, time/batch = 0.014
>> sample mode:
The sluck stabe of Chiec.
Why, then a pary; he part.


----------------------------------
